<font face="Calibri" size="2"> <i>Open SAR Toolkit - Tutorial 4c, version 1.2, August 2020. Andreas Vollrath, ESA/ESRIN phi-lab</i>
</font>

![title](https://raw.githubusercontent.com/ESA-PhiLab/OpenSarToolkit/main/docs/source/_images/header_image.PNG)

--------

# OST Tutorial IV - C
## Create country-wide mosaic time-series and timescan. Introduction to GRD Batch processing part III.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ESA-PhiLab/OST_Notebooks/blob/master/4c%20-%20Sentinel-1%20GRD%20Batch%20-%20Timescan.ipynb)

--------

**Short description**

This notebook is very similar to the Tutorial IVa, with the difference that you will process GRD data over a larger area and create time-series and timescan mosaics. It therefore represents the workflow for large-scale processing.

--------

**Requirements**

- a PC/Mac with at least 16GB of RAM
- about 150GB of free disk space
- a Copernicus Open Data Hub user account, valid for at least 7 days (https://scihub.copernicus.eu)
--------

**NOTE:** all cells that have an * after its number can be executed without changing any code. 

### 0\* - Install OST and dependencies 

**NOTE:** Applies only if you haven't fully installed OST and its dependencies yet, e.g. on Google Colab, so uncomment the lines in this case.

In [ ]:
# !apt-get -y install wget
# !wget https://raw.githubusercontent.com/ESA-PhiLab/OST_Notebooks/master/install_ost.sh
# !bash install_ost.sh

### 1* - Import of Libraries

In this step we import some standard python libraries for OS independent path handling as well as the *Sentinel1_GRDBatch* class thta handles the full workflow from search, download and processing of multiple GRD scenes. In addition, the OST helper module *vector* is loaded to create an AOI based on Point coordinates, and the *raster* module for creating a time-series animation.

In [ ]:
from pathlib import Path
from pprint import pprint

from ost import Sentinel1Batch
from ost.helpers import vector, raster

### 2* - Set up the project 

Here you going to initialize the *Sentinel1_GRDBatch* class by determining the project folder, the AOI and the start and end date. Since you should be already familiar with the *search* and *refine* functions, we execute them within the same cell.

In [ ]:
# define a project directory
home = Path.home()
# create a processing directory
project_dir = home / "OST_Tutorials" / "Tutorial_4c"

# define aoi with helper function, i.e. get a buffered area around point coordinates
aoi = "IRL"

# define the start and end date
start = "2019-02-01"
end = "2019-04-30"

# initialize the class to s1_grd instance
s1_grd = Sentinel1Batch(project_dir=project_dir, aoi=aoi, start=start, end=end, product_type="GRD")

# trigger the search
s1_grd.search()

# optional: once you did the search the first time, you can load
# the full inventory uncommenting the follwoing 2 lines
# and commenting the search command
# s1_grd.inventory_file = s1_grd.inventory_dir/"full.inventory.gpkg"
# s1_grd.read_inventory()

# do the refinement
s1_grd.refine_inventory()

### 3* - Plot refined data inventory

Here you will visualize the resultant dataframes from the refined search inventory based on the product key.

In [ ]:
# ---------------------------------------------------
# for plotting purposes we use this iPython magic
%matplotlib inline
%pylab inline
pylab.rcParams["figure.figsize"] = (19, 19)
# ---------------------------------------------------

# search command
key = "ASCENDING_VVVH"
nb_frames = len(s1_grd.refined_inventory_dict[key])
print(f"Our refined inventory holds {nb_frames} frames to process.")

# we plot the full Inventory on a map
s1_grd.plot_inventory(s1_grd.refined_inventory_dict[key], transparency=0.1)

### 4* - Download of GRD scenes

As already shown in Tutorial II, you will download the scenes based on the refined inventory dataframe for the respective produckt key.

In [ ]:
s1_grd.download(s1_grd.refined_inventory_dict[key])

### 5* - Set ARD parameters

Similar to the *Sentinel1-Scene* class (Tutorial I and III), the *Sentinel1-GRDBatch* class handles the defintion of ARD types in a hierarchical dictionary structure. You can use the same types and steps to customize as for the *Sentinel1-Scene* class.

In [ ]:
# single scene ARD parameters
s1_grd.ard_parameters["single_ARD"]["resolution"] = 50
s1_grd.ard_parameters["single_ARD"]["product_type"] = "GTC-gamma0"
s1_grd.ard_parameters["single_ARD"]["create_ls_mask"] = False

# time-series ARD
s1_grd.ard_parameters["time-series_ARD"]["remove_mt_speckle"] = False

# our borders for Ireland are quite rough. We therefore won't clip the final mosaics
s1_grd.ard_parameters["mosaic"]["cut_to_aoi"] = False

# s1_grd.config_dict['temp_dir'] = '/ram'
pprint(s1_grd.ard_parameters)

### 6* - Run the batch routine

To process all the data, including time-series and timescans is as easy as one command. All the complexity is handled behind, and you just have to wait, since processing can take quite a while.

In [ ]:
s1_grd.grds_to_ards(
    inventory_df=s1_grd.refined_inventory_dict[key],
    timeseries=True,
    timescan=True,
    overwrite=False,
)

### 7* - Create a time-series animation

For interactive presentations it is nice to have animated "movies". The following command allows you to create animated time-series of oyur processed data.

In [ ]:
# define Time-series folder
ts_folder = s1_grd.processing_dir / "23" / "Timeseries"

# create the animation
raster.create_timeseries_animation(
    timeseries_folder=ts_folder,
    product_list=["bs.VV", "bs.VH"],
    out_folder=s1_grd.processing_dir,
    shrink_factor=10,
    add_dates=True,
)